In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/c

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tokens").getOrCreate()

In [ ]:
from pyspark.ml.feature import Tokenizer

In [ ]:
# Create sample Dataframe
dataframe = spark.createDataFrame([(0, "Spark"),
                                   (1, "Learning Spark"),
                                   (2, "Spark is faster than hadoop")], ["id", "sentence"])

In [ ]:
dataframe.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|               Spark|
|  1|      Learning Spark|
|  2|Spark is faster t...|
+---+--------------------+



In [ ]:
#Tokenize sentences
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenizer

Tokenizer_23b95cd9c215

In [ ]:
tokenizer_df = tokenizer.transform(dataframe)
tokenizer_df.show(truncate=False)

+---+---------------------------+---------------------------------+
|id |sentence                   |words                            |
+---+---------------------------+---------------------------------+
|0  |Spark                      |[spark]                          |
|1  |Learning Spark             |[learning, spark]                |
|2  |Spark is faster than hadoop|[spark, is, faster, than, hadoop]|
+---+---------------------------+---------------------------------+



In [ ]:
# Create a function to return the length of a list
def word_list_length(word_list):
    return len(word_list)

In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [ ]:
# Create a user defined token
count_tokens = udf(word_list_length, IntegerType())

In [ ]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

In [ ]:
tokenized_df = tokenizer.transform(dataframe)

In [ ]:
tokenized_df.withColumn("tokens", count_tokens(col("words"))).show(truncate=False)

+---+---------------------------+---------------------------------+------+
|id |sentence                   |words                            |tokens|
+---+---------------------------+---------------------------------+------+
|0  |Spark                      |[spark]                          |1     |
|1  |Learning Spark             |[learning, spark]                |2     |
|2  |Spark is faster than hadoop|[spark, is, faster, than, hadoop]|5     |
+---+---------------------------+---------------------------------+------+

